In [1]:
import torch
from torch import nn as nn
from torch.nn import functional as F
import numpy as np
from tqdm import tqdm 

In [9]:
# Device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
# Hyperparameters
block_size = 8
batch_size = 4
max_iterations = 1000
lr = 3e-4
eval_iterations = 250
dropout = 0.2
num_embd = 384 
num_layers = 4

In [3]:
with open("content.txt", 'r', encoding='utf-8') as f:
    for line in f:
        text = f.read()

characters = sorted(set(text))
vocab_size = len(characters)

In [4]:
# Creating a mapping from unique characters to indices
string_to_int = { char:index for index, char in enumerate(characters)}
int_to_string = { index:char for index, char in enumerate(characters)}

# Encoding the text
encode = lambda s : [string_to_int[c] for c in s]
decode = lambda c : ''.join([int_to_string[i] for i in c])

# Turn into tensors 
data = torch.tensor(encode(text), dtype=torch.long)


In [5]:
split = int(len(data) * 0.8) # Hyperparameter

train_data = data[:split].to(device)
test_data = data[split:].to(device)

def get_batch(split: str = "train"):
    """Returns a random batch from the data"""
    data = train_data if split == "train" else test_data
    start_index = torch.randint(len(data) - block_size, (batch_size,))
    # print(start_index)
    x = torch.stack([data[idx:idx+block_size] for idx in start_index])
    y = torch.stack([data[idx+1:idx+block_size+1] for idx in start_index])
    x, y = x.to(device), y.to(device) # Make sure data is on the right device
    return x, y

In [44]:
x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"When input is {context}, the target is {target}")

When input is tensor([22], device='cuda:0'), the target is 47
When input is tensor([22, 47], device='cuda:0'), the target is 56
When input is tensor([22, 47, 56], device='cuda:0'), the target is 1
When input is tensor([22, 47, 56,  1], device='cuda:0'), the target is 61
When input is tensor([22, 47, 56,  1, 61], device='cuda:0'), the target is 51
When input is tensor([22, 47, 56,  1, 61, 51], device='cuda:0'), the target is 49
When input is tensor([22, 47, 56,  1, 61, 51, 49], device='cuda:0'), the target is 56
When input is tensor([22, 47, 56,  1, 61, 51, 49, 56], device='cuda:0'), the target is 43


In [6]:
def estimate_loss(model: nn.Module):
    out = {}
    with torch.inference_mode():
        model.eval()
        for split in ['train', 'val']:
            losses = torch.zeros(eval_iterations)
            for k in range(eval_iterations):
                X, Y = get_batch(split)
                logits, loss = model(X, Y)
                losses[k] = loss.item()
            out[split] = losses.mean()
        model.train()
    return out 

In [14]:
class Head(nn.Module):
    """One head of self-attention"""
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(num_embd, head_size, bias=False)
        self.query = nn.Linear(num_embd, head_size, bias=False)
        self.value = nn.Linear(num_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.droupout = nn.Dropout(dropout)

    def forward(self, x):
        # Input shape (Batch, time-step, channels)
        # Output shape: (Batch, time-step, head size)
        B, T, C = x.shape
        # Calling the transformations, but different ones for each
        k = self.key(x) # (B, T, head size)
        q = self.query(x) # (B, T, head size)
        # Compute attention scores
        weight = q @ k.transpose(-2, -1) * k.shape[-1]**-0.5 # (B, T, T)
        weight = weight.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # Mask out upper triangular 
        weight = F.softmax(weight, dim=-1) # (B, T, T)
        weight = self.dropout(weight)
        # Apply attention
        v = self.value(x) # (B, T, head size)
        out = weight @ v # (B, T, head size)
        return out

In [12]:
class MultiHeadAttention(nn.Module):
    """ Multiple heads of self-attention in parallel """
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(in_features=head_size * num_heads, out_features=num_embd)
        self.dropout = nn.Dropout(dropout) 
    
    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1) # (B, T, F) where F = head_size * num_heads
        out = self.dropout(self.proj(out)) # (B, T, E)
        return out

In [11]:
class FeedForward(nn.Module):
    """A simple feed forward with linear layers and ReLU activations"""
    def __init__(self, num_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_features=num_embd, out_features=4*num_embd),
            nn.ReLU(),
            nn.Linear(in_features=4*num_embd, out_features=num_embd),
            nn.Dropout(dropout) # Dropout to prevent overfitting
        )
    
    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """Implementation of a Transformer block"""
    def __init__(self, num_embd, num_head):
        super().__init__()
        head_size = num_embd // num_head 
        self.self_attention = MultiHeadAttention(num_embd, head_size)
        self.feed_forward = FeedForward(num_embd)
        self.ln1 = nn.LayerNorm(num_embd)
        self.ln2 = nn.LayerNorm(num_embd)

    def forward(self, x):
        y = self.self_attention(x)
        x = self.ln1(x + y)
        y = self.feed_forward(x)
        x = self.ln2(x + y)
        return x


class GPTModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, num_embd)
        self.position_embedding_table = nn.Embedding(block_size, num_embd)
        self.blocks = nn.Sequential(*[Block(num_embd, num_head=num_head) for _ in range(num_layers)])
        self.ln_f = nn.LayerNorm(num_embd) # Layer normalization
        self.lm_head = nn.Linear(num_embd, vocab_size) # Language modeling head, i.e. the weights to the logits

        self.apply(self._init_weights)

        def _init_weights(self, module):
            if isinstance(module, nn.Linear):
                nn.init.normal_(module.weight, mean=0.0, std=0.02)
                if module.bias is not None:
                    nn.init.zeros_(module.bias)
            elif isinstance(module, nn.Embedding):
                nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, index, targets=None): 
        logits = self.token_embedding_table(index)

        # Feed the sequence through the transformer blocks
        token_embd = self.token_embedding_table(index)
        position_embd = self.position_embedding_table(torch.arange(T, device=device)) # (T, C)
        x = token_embd + position_embd # (B, T, C)
        x = self.blocks(x) # (B, T, C)
        x = self.ln_f(x) # (B, T, C)
        logits = self.lm_head(x) # (B, T, vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape # batch_size, time, vocab_size
            logits = logits.view(B*T, C) # Flatten the batch and time dimensions
            targets = targets.view(B*T) # Flatten the batch and time dimensions 
            loss = F.cross_entropy(logits, targets) # Calculate the loss 

        return logits, loss

    # Generate new content
    def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            # Make predictions
            logits, loss = self.forward(index)
            # Get the last time step
            logits = logits[:, -1, :] # (B, C)
            # Turn into probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # Sample from the distribution or take the most likely
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # Append to the sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

In [63]:
model_0 = BigramLanguageModel(vocab_size).to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(model_0.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


JgFJ;VøzeeAYëëL:KmJoS6cgrp1vPéz9ü’Ö0A4q’CLDCJFfåOn6–9ADCTpyUqSm5Öpc39 :cüG41p1EtSwUzÖ–KpXFlIGr ti7VWmUüpYhLDg.mLl’Lä?…—SEHmIeW9ükVuUz9Nñu,hag
U5hl,uFIKwO30v2oNñæWYñefsF?3?…rp–LéiwjOPføoD…HacFMOæÅæ
üB8WYg:KDtrffO?4wÄNwW-Op’v,pVdëIx3øC1zj0XTp9
hL;—ø–Pä;Uns;fC0i ÄfoB–F?Äw6D
M?J,p1qM—-4Eåünmd?KRu2O-29XF
étåti:éCqÅÖjëIvd8hüctVu2Y9!öXøWYbe?Gcæu14IkñYb9vlS3e86K6Gäu2O-olSkYg7—tTuWNñRXl8D3LnJAj3!F?D,th-… jO,s;kcI
kG:—ÖjTëL4LéäågXAWYYreu,KtP!;’.MOP76R’n.Öë::kYñLXvö2!NbfPyoB p:p1kNruUAYvYr’vvd FBfStyüCcH;!


In [106]:
optimizer = torch.optim.AdamW(model_0.parameters(), lr=lr)

for iter in tqdm(range(max_iterations)):
    if iter % eval_iterations == 0:
        losses = estimate_loss(model_0)
        print(f'Step: {iter} | Train Loss: {losses["train"]:.4f} | Val Loss: {losses["val"]:.4f}')
    # Sample a batch of data
    xb, yb = get_batch("train")

    # Calculate the loss 
    logits, loss = model_0(xb, yb)
    optimizer.zero_grad(set_to_none=True) # Clear the gradients
    loss.backward() # Calculate the gradients
    optimizer.step() # Update the weights

print(loss.item())

  6%|▌         | 56/1000 [00:00<00:08, 112.97it/s]

Step: 0 | Train Loss: 2.4214 | Val Loss: 2.4200


 32%|███▏      | 316/1000 [00:01<00:02, 272.81it/s]

Step: 250 | Train Loss: 2.3808 | Val Loss: 2.4350


 56%|█████▌    | 557/1000 [00:02<00:01, 303.85it/s]

Step: 500 | Train Loss: 2.3753 | Val Loss: 2.4320


 84%|████████▍ | 838/1000 [00:03<00:00, 326.92it/s]

Step: 750 | Train Loss: 2.3823 | Val Loss: 2.4134


100%|██████████| 1000/1000 [00:03<00:00, 304.31it/s]

2.164550542831421


In [111]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(model_0.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


inochartttist sku j, fr ge sedene och venyme het Dad, st ut henslynkych ägtn, hadendenonyn i m met lvi a ommllllku usmesfr.
MHa ktslllättadesäril dendarrtilt bynndde. hofanomåga hate Derdanäst le Sår h fi ranörn, m h vi ivelaskenängen. kadgapa bli h Dede gå om e hart. han hut Ha inar föv an stt h sspt dag ha, imetonst s.
De h hott ag ne jkadjuttändedjäm. mislat.
Pkiga mar Därar mdlesk män t.
Hon, ssvrleryt vi br sfr kt hörbän anätevetrskage lam spå fonie de he st vadenngäru d, s h voch sk, aren 
